In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()
    
print("replocas: ", strategy.num_replicas_in_sync)

In [ ]:
# Get the GCS path
GCS_PATH = KaggleDatasets().get_gcs_path('plant-pathology-2021-fgvc8')
print(GCS_PATH)

train_path = GCS_PATH + '/train_images/'
print(train_path)

* Labels code as multi-label classification

In [ ]:
train_df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df['labels'].value_counts().to_frame()

In [ ]:
labels2id = {
    'scab': 0,
    'healthy': 1,
    'frog_eye_leaf_spot': 2,
    'rust': 3,
    'complex': 4,
    'powdery_mildew': 5
}

id2labels = {v:k for k,v in labels2id.items()}

label_classes = labels2id.keys()

def label_encoder(x : str):
    return [1 if label in x.split(' ') else 0 for label in label_classes]

train_df['labels'] = train_df['labels'].map(label_encoder)

In [ ]:
train_df.head()

In [ ]:
image_size = [600,600]
batch_size = 16 * strategy.num_replicas_in_sync
channels = 3
seed = 2021
num_classes = len(label_classes)
AUTOTUNE = tf.data.experimental.AUTOTUNE

* The training dataset, validation dataset and test dataset are divided according to 8:1:1

In [ ]:
image_file_path = np.array([train_path + i for i in train_df['image'].to_list()])
labels = train_df['labels'].to_numpy()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

train_val_sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=seed)

for train_index, val_index in train_val_sss.split(image_file_path, labels):
    train_path, val_path = image_file_path[train_index],image_file_path[val_index]
    train_labels, val_labels = labels[train_index],labels[val_index]

In [ ]:
print('train : ',len(train_path),'---',len(train_labels))
print('val : ',len(val_path),'---',len(val_labels))

* convert label's dtype from list to tensor

In [ ]:
train_labels[:3]

In [ ]:
train_labels = [tf.constant(x) for x in train_labels]
val_labels = [tf.constant(x) for x in val_labels]

In [ ]:
train_labels[:3]

* Process image

In [ ]:
def random_erasing(img, sl=0.1, sh=0.2, rl=0.4, p=0.3):
    h = tf.shape(img)[0]
    w = tf.shape(img)[1]
    c = tf.shape(img)[2]
    origin_area = tf.cast(h*w, tf.float32)

    e_size_l = tf.cast(tf.round(tf.sqrt(origin_area * sl * rl)), tf.int32)
    e_size_h = tf.cast(tf.round(tf.sqrt(origin_area * sh / rl)), tf.int32)

    e_height_h = tf.minimum(e_size_h, h)
    e_width_h = tf.minimum(e_size_h, w)

    erase_height = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_height_h, dtype=tf.int32)
    erase_width = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_width_h, dtype=tf.int32)

    erase_area = tf.zeros(shape=[erase_height, erase_width, c])
    erase_area = tf.cast(erase_area, tf.uint8)

    pad_h = h - erase_height
    pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32)
    pad_bottom = pad_h - pad_top

    pad_w = w - erase_width
    pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32)
    pad_right = pad_w - pad_left

    erase_mask = tf.pad([erase_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
    erase_mask = tf.squeeze(erase_mask, axis=0)
    erased_img = tf.multiply(tf.cast(img,tf.float32), tf.cast(erase_mask, tf.float32))

    return tf.cond(tf.random.uniform([], 0, 1) > p, lambda: tf.cast(img, img.dtype), lambda:  tf.cast(erased_img, img.dtype))

In [ ]:
def load_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_jpeg(image, channels=channels)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, image_size)
    return image, label


augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(factor=0.02),
    layers.experimental.preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2),
])

def augment(image, label):
    image = tf.expand_dims(image, axis=0)
    image = augmentation(image)[0]
    #image = tf.image.random_brightness(image, 0.2)
    #image = tf.image.random_contrast(image, 0.5, 2.0)
    #image = tf.image.random_saturation(image, 0.75, 1.25)
    #image = tf.image.random_hue(image, 0.1)
    image = random_erasing(image)
    return image,label

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_path,train_labels))
train_ds = train_ds.map(load_image, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(augment, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache().shuffle(2048).batch(batch_size).prefetch(AUTOTUNE)


val_ds = tf.data.Dataset.from_tensor_slices((val_path,val_labels))
val_ds = val_ds.map(load_image, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.cache().batch(batch_size).prefetch(AUTOTUNE)

* view some pictures

In [ ]:
image,_ = next(iter(train_ds))

plt.figure(figsize=(20,20))

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow((image[i].numpy() * 255).astype('uint8'))
    plt.axis('off')

plt.show()

In [ ]:
def plot_learning_curve(history):
    history = history.history
    metrics_names = ['loss','accuracy','precision','recall','f1_score']
    plt.figure(figsize=(8, 35))
    for i,name in enumerate(metrics_names):
        plt.subplot(len(metrics_names),1,i+1)
        plt.plot(history[name], label='training '+name)
        plt.plot(history['val_'+name], label='validation '+name)
        plt.legend(loc='lower right')
        plt.ylabel(name)
        plt.ylim([0,1])
        plt.title('training and validation '+name)
    plt.show()

# EfficientNetB4

In [ ]:
with strategy.scope():
    DenseNet = keras.applications.EfficientNetB4(include_top=False)
    model = keras.Sequential()
    model.add(layers.Input(shape=[*image_size,channels]))
    model.add(DenseNet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(num_classes,activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.05),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tfa.metrics.F1Score(num_classes=num_classes,average='macro',name='f1_score')
        ]
     )
    
    callbacks = [
        # keras.callbacks.EarlyStopping(patience=10,verbose=1,restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=4,verbose=1,min_delta=0.00001,
                                         monitor='val_f1_score',mode='max'),
        keras.callbacks.ModelCheckpoint('EfficientNetB4-600.h5',monitor='val_f1_score',mode='max',
                                        save_best_only=True,verbose=1)
    ]

    history = model.fit(train_ds,batch_size=batch_size,epochs=100,
            validation_data=val_ds,callbacks=callbacks)

In [ ]:
plot_learning_curve(history)

# ResNet152V2

In [ ]:
with strategy.scope():
    ResNet = keras.applications.ResNet152V2(include_top=False)
    model = keras.Sequential()
    model.add(layers.Input(shape=[*image_size,channels]))
    model.add(ResNet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(num_classes,activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.05),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tfa.metrics.F1Score(num_classes=num_classes,average='macro',name='f1_score')
        ]
     )
    
    callbacks = [
        # keras.callbacks.EarlyStopping(patience=10,verbose=1,restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=4,verbose=1, min_delta=0.00001,
                                         monitor='val_f1_score',mode='max'),
        keras.callbacks.ModelCheckpoint('ResNet152V2-600.h5',monitor='val_f1_score',mode='max',
                                        save_best_only=True,verbose=1)
    ]

    history = model.fit(train_ds,batch_size=batch_size,epochs=100,
            validation_data=val_ds,callbacks=callbacks)

In [ ]:
plot_learning_curve(history)

# Submit

In [ ]:
def load_predict_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_jpeg(image, channels=channels)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, image_size)
    image = tf.expand_dims(image,axis=0)
    return image

In [ ]:
test_path = '../input/plant-pathology-2021-fgvc8/test_images/'

submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
for model_name in ['EfficientNetB4-600.h5','ResNet152V2-600.h5']:
    model = keras.models.load_model(model_name)
    
    for row in submission.index:

        image = load_predict_image(test_path+submission.loc[row,'image'])
        predict = model.predict(image)[0]
        predict = [1 if i>0.5 else 0 for i in predict]
        result = []
        for i,j in enumerate(predict):
            if j:
                result.append(id2labels.get(i))
        result = ' '.join(result)
        submission.loc[row,'labels'] = result
    
    submission.to_csv(model_name[:-6]+'submission.csv',index=False)